In [1]:
import cv2
import os 
import random
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Conv2D,Dense,MaxPool2D,Input,Flatten
import tensorflow as tf

In [2]:
class L1Dist(Layer):
    
    def __init__(self,**kwargs):
        super().__init__()
        
    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding-validation_embedding)

In [3]:
model=tf.keras.models.load_model('siamesemodel.h5',
        custom_objects={'L1Dist':L1Dist,'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [4]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_image (InputLayer)    [(None, 100, 100, 3)]        0         []                            
                                                                                                  
 validation_img (InputLayer  [(None, 100, 100, 3)]        0         []                            
 )                                                                                                
                                                                                                  
 embedding (Functional)      (None, 4096)                 3896044   ['input_image[0][0]',         
                                                          8          'validation_img[0][0]']      
                                                                                     

In [5]:
def verify(model,detection_threshold,verification_threshold):
    results=[]
    image_path=[]
    for image in os.listdir(os.path.join('application_data','verification_image')):
        
        input_img=preprocess(os.path.join('application_data','input_image','input_image.jpg'))
        validation_img=preprocess(os.path.join('application_data','verification_image',image))
        validation_img_path=os.path.join('application_data','verification_image',image)
        
        result=model.predict(list(np.expand_dims([input_img,validation_img],axis=1)))
        results.append(result)
        image_path.append(validation_img_path)
        
    detection=np.sum(np.array(results)>detection_threshold)
    verification=detection/len(os.listdir(os.path.join('application_data','verification_image')))
    verified=verification>verification_threshold
    
    return results,detection,image_path

In [6]:
def preprocess(file_path):
    byte_img=tf.io.read_file(file_path)
    img=tf.io.decode_jpeg(byte_img)
    img=tf.image.resize(img,(100,100))
    img=img/255.0
    return img

In [17]:
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 250)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 250)
while cap.isOpened():
    ret,frame=cap.read()
    frame = cv2.resize(frame, (250, 250))
    frame=cv2.flip(frame,1)
    
    cv2.imshow('verification',frame)
    
    if cv2.waitKey(1) & 0XFF==ord('v'):
        cv2.imwrite(os.path.join('application_data','input_image','input_image.jpg'),frame)
        results,verified,image_path=verify(model,0.7,0.6)
        print(verified)
    
    if cv2.waitKey(1) & 0XFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 9ms/step
3


In [18]:
max_index = np.argmax([arr[0, 0] for arr in results])

In [19]:
max_index

1

In [20]:
image_path[max_index].split('/')[2]

'22124004.jpg'

In [21]:
results

[array([[0.9998754]], dtype=float32),
 array([[0.9999268]], dtype=float32),
 array([[0.45581213]], dtype=float32),
 array([[0.9995351]], dtype=float32)]

In [22]:
image_path

['application_data/verification_image/22124006.jpeg',
 'application_data/verification_image/22124004.jpg',
 'application_data/verification_image/22124002.jpg',
 'application_data/verification_image/22124007.jpeg']